In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
tables = ['table_3.csv', 'table_17.csv', 'table_18.csv', 'table_19.csv']
tables

['table_3.csv', 'table_17.csv', 'table_18.csv', 'table_19.csv']

In [5]:
df1 = pd.read_csv(tables[0])
df2 = pd.read_csv(tables[1])
df3 = pd.read_csv(tables[2])
df4 = pd.read_csv(tables[3])

In [26]:
def check_column_equal(df1, df2):
    df1_columns = df1.columns.tolist()
    df2_columns = df2.columns.tolist()
    
    number_of_columns1 = len(df1_columns)
    number_of_columns2 = len(df2_columns)
    
    print("number_of_columns1: ", number_of_columns1)
    print("number_of_columns2: ", number_of_columns2)
    
    if number_of_columns1 != number_of_columns2:
        return False
    
    if df1_columns == df2_columns:
        return True
    else:
        return False
    
def solve_non_header_table(df, headers):
    temp_firstrow = pd.DataFrame([list(df.columns)], columns=headers)
    df.columns = headers

    # insert temp_data in to the first row
    df = pd.concat([pd.DataFrame(temp_firstrow, columns=headers), df], ignore_index=True)
    df = df.reset_index(drop=True)
    return df

def concat_df(df1, df2):
    df1_columns = df1.columns.tolist()
    df2_columns = df2.columns.tolist()
    if df1_columns == df2_columns:
        return pd.concat([df1, df2], ignore_index=True)
    else:
        return None

In [34]:
def detect_csv_header(file_path: str) -> bool:
    """
    Detect if a CSV file has no header using csv.Sniffer.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        bool: True if the CSV likely has NO header, False otherwise.
    """
    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as csv_file:
            # Read sample data (first 10 lines) for analysis
            sample_data = "".join(next(csv_file) for _ in range(10))
            
            if not sample_data.strip():
                return True  # Empty file -> no header

            # Use CSV Sniffer to detect header
            sniffer = csv.Sniffer()
            has_header = sniffer.has_header(sample_data)
            return not has_header
            
    except FileNotFoundError:
        print(f"Error: File not found '{file_path}'")
        return False
    except csv.Error:
        # If Sniffer fails to analyze, assume no header
        return True
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False